# TP3: Detector de SPAM

## Integrantes

- Nicolás Rodriguez da Cruz
- Francisco Cofré
- Gaspar Acevedo Zain
- Juan Chunga
- Rodrigo Nicolás Lauro

### Imports y carga del dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Scikit-learn ofrece una variedad de modelos Naive Bayes. Para este problema, utilizamos MultinomialNB, que es adecuado para datos de conteo como este.
from sklearn.naive_bayes import MultinomialNB   

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
dataset = pd.read_csv("dataset/spambase.csv")

In [3]:
dataset.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,spam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,104.553358,213.014345,280.656379,65.424908,312.222995,95.900891,114.207564,105.294501,90.067377,239.413171,...,179.823734,5.444469,31.869159,38.574440,139.030428,16.975875,269.068898,75.810259,44.237992,0.394045
std,305.357562,1290.574888,504.142884,1395.151370,672.511666,273.824083,391.440302,401.071452,278.615864,644.755399,...,911.118627,76.274271,285.734646,243.470469,270.355374,109.394164,815.669848,245.879440,429.341596,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,65.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,420.000000,0.000000,380.000000,0.000000,0.000000,0.000000,0.000000,160.000000,...,0.000000,0.000000,0.000000,0.000000,188.000000,0.000000,315.000000,52.000000,0.000000,1.000000
max,4540.000000,14280.000000,5100.000000,42810.000000,10000.000000,5880.000000,7270.000000,11110.000000,5260.000000,18180.000000,...,22050.000000,2170.000000,10000.000000,4385.000000,9752.000000,4081.000000,32478.000000,6003.000000,19829.000000,1.000000


### 1. ¿Cuáles son las 10 palabras o símbolos más frecuentes en correos SPAM y en correos NO SPAM?

1. ¿Hay palabras o símbolos en común?
1. ¿Alguna resulta llamativa?

#### Resolución

*10 palabras más repetidas en correos **NO spam***
| Palabra | Frecuencia |
| --- | --- |
| you | $3541702$ |
| george | $3527559$ |
| hp | $2496576$|
| will | $1495268$ |
| your | $1223098$ |
| hpl | $1204398$ |
| re | $1159138$ |
| edu | $800669$ |
| address | $681569$ |
| meeting | $604460$ |

*10 palabras más repetidas en correos **spam***
| Palabra | Frecuencia |
| --- | --- |
| you | $4105599$ |
| your | $2502597$ |
| will | $997100$ |
| free | $939790$ |
| our | $931799$ |
| ! | $931352$ |
| all | $732080$ |
| mail | $635470$ |
| email | $578759$ |
| business | $52125$ |

##### 1.1 ¿Hay palabras o símbolos en común?

Si, entre las diez palabras/símbolos más frecuentes en correos marcados como Spam y NO Spam hay tres palabras repetidas:

| Palabra | Frecuencia NO Spam | Frecuencia Spam |
| --- | --- | --- |
| you | $3541702$ | $4105599$ |
| your | $1223098$ | $2502597$ |
| will | $1495268$ | $997100$ |

Como se puede observar, la palabra `you` aparece $3.541.702$ de veces en correos NO Spam, pero se repite más veces en correos Spam, con un total de $4.105.599$.
Con la palabra `your` sucede algo similar: aparece $1.223.098$ veces en correos No Spam, pero en correos Spam la cantidad de veces es mayor, con un total de $2.502.597$.
Por último, la palabra `will` aparece más veces en correos No Spam: $1.495.268$, contra $997.100$ veces en correos Spam.

##### 1.2 ¿Alguna resulta llamativa?

Las que nos resultan llamativas de correos NO Spam son las palabras `hp` (aparece $2.496.576$), `hpl` ($1.204.398$), y `re` ($1.159.138$). Si bien, desconocemos el significado de las dos primeras, la tercer palabra (`re`) puede referirse al termino que aparece cuando el email es una respuesta o *reply* de una cadena de mensajes.

En cuanto a las palabras y símbolos llamativos de correos Spam, notamos que la palabra `free` (gratis en inglés) aparece $939.790$ de veces, mientras que el símbolo de explamación `!` aparece $931.352$ de veces. Esto puede deberse a que son palabras/símbolos utilizados para llamar la atención del receptor.

#### Código para la resolución del punto 1

In [57]:
# Primero, agrupamos por la columna "spam" y sumamos la cantidad de datos para cada columna
spam_col = "spam"
group_by = dataset.groupby(by=spam_col, as_index=False).sum()

# Subdividimos en Spam y No Spam, para facilitar el análisis
group_by_no_spam = group_by[group_by[spam_col]==0]
group_by_spam = group_by[group_by[spam_col]==1]

# Usando pandas.melt (https://pandas.pydata.org/docs/reference/api/pandas.melt.html), hacemos un pivot, y luego ordenamos por la columna "Values" de manera descendente
# NOTE: hacemos drop de la columna "spam" para que no aparezca entre los resultados
value_col = "value"
spam_ordered = pd.melt(group_by_spam.drop(columns=spam_col)).sort_values(by=value_col, ascending=False)
no_spam_ordered = pd.melt(group_by_no_spam.drop(columns=spam_col)).sort_values(by=value_col, ascending=False)

In [61]:
# definimos la cantidad de símbolos a mostrar
simbols_to_show = 10

In [62]:
no_spam_ordered.head(n=simbols_to_show)

,variable,value
18,word_freq_you,3541702
26,word_freq_george,3527559
24,word_freq_hp,2496576
11,word_freq_will,1495268
20,word_freq_your,1223098
25,word_freq_hpl,1204398
44,word_freq_re,1159138
45,word_freq_edu,800669
1,word_freq_address,681569
41,word_freq_meeting,604460


In [56]:
spam_ordered.head(n=simbols_to_show)

,variable,value
18,word_freq_you,4105599
20,word_freq_your,2502597
11,word_freq_will,997100
15,word_freq_free,939790
4,word_freq_our,931799
51,char_freq_!,931352
2,word_freq_all,732080
9,word_freq_mail,635470
17,word_freq_email,578759
16,word_freq_business,521250


### 2. Separe el conjunto de datos en un conjunto de entrenamiento (70%) y uno de prueba (30%).

#### Resolución

Antes de hacer la división en conjuntos de entrenamiento y prueba con valores 70%-30% respectivamente, debemos analizar si nuestra variable objetivo `Spam` está balanceada.
Luego de analizar la cantidad de correos maracados como Spam o No Spam, encontramos que no están balanceados, ya que hay $1813$ Spam y $2788$ No Spam.

Por eso, al realizar la división en train-test, debemos utilizar el parámetro [stratify](https://scikit-learn.org/stable/modules/cross_validation.html#stratification) de la funcion [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

De esta manera, obtenemos:
- `Conjunto de entrenamiento`: 1269 Spam y 1951 No Spam
- `Conjunto de prueba`: 544 Spam y 837 No Spam

#### Código para la resolución del punto 2

In [67]:
# Primero, obtenemos la cantidad de registros marcados como Spam y NO Spam, para determinar si las clases están balanceadas
no_spam_data = dataset[dataset[spam_col]==0]
spam_data = dataset[dataset[spam_col]==1]

print(f"Cantidad de correos marcados como SPAM: {len(spam_data)}")
print(f"Cantidad de correos marcados como No SPAM: {len(no_spam_data)}")

Cantidad de correos marcados como SPAM: 1813
Cantidad de correos marcados como No SPAM: 2788


In [80]:
# Debido a que las clases están desbalanceadas (1813 SPAM y 2788 No SPAM), al dividir en Train y Test debemos hacer un stratify

X = dataset.drop(columns=spam_col)
y = dataset[spam_col]

random_state = 100019
test_size = 0.3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = random_state, stratify=y)

In [81]:
print(f"Cantidad de correos SPAM en train: {len(y_train[y_train==1])}")
print(f"Cantidad de correos No SPAM en train: {len(y_train[y_train==0])}")

print(f"Cantidad de correos SPAM en test: {len(y_test[y_test==1])}")
print(f"Cantidad de correos No SPAM en test: {len(y_test[y_test==0])}")

Cantidad de correos SPAM en train: 1269
Cantidad de correos No SPAM en train: 1951
Cantidad de correos SPAM en test: 544
Cantidad de correos No SPAM en test: 837


### 3. Utilizando un **clasificador de Bayes ingenuo**, entrene el modelo con el conjunto de entrenamiento.

### 4. Utilizando un **clasificador de Regresión Logística**, entrene el modelo con el conjunto de entrenamiento (en este caso, normalice los datos).

### 5. Calcule la **matriz de confusión** en el conjunto de prueba para ambos modelos.

1. ¿Qué tipo de error comete más cada modelo?
1. ¿Cuál de los dos tipos de error considera más importante en este problema?

### 6. Calcule las **métricas de precisión (precision) y recuperación (recall)** para ambos modelos.

1. ¿Cuál es el mejor modelo según cada métrica?
1. ¿Cómo se relacionan estas métricas con los errores analizados en el punto anterior? Fundamente su respuesta.

### 7. Obtenga la **curva ROC y el AUC (Área Bajo la Curva ROC)** de ambos modelos.